# Goal

### Generating Dungeon's And Dragons Backstory

Markov Chain relies heavily on the dataset it was trained on without understanding the context, thus is perfect to generate fantasy/artificial backstory. Markov Chain model can generate backstories for DND characters faster than other Deep Learning models as it will just bluff up stuff using the provided tokens.

# Requirement

### The Dataset

Generated using:
https://capitalizemytitle.com/character-generator/dnd-backstory/

The parameters as follows:

| Parameter         | Value           |
|-------------------|-----------------|
| Gender            | any             |
| Character Name    | X               |
| Character Details | (not specified) |
| DnD Class         | any             |
| DnD Race          | any             |

The generator was run 5 times with each generation results in 3 paragraphs (3 lines), with total of 15 lines.
It is then combined in [dataset/backstory-combined.txt](dataset/backstory-combined.txt).


### Importing the package

In [9]:
import markovify

### Reading the train dataset

In [10]:
with open("dataset/backstory-combined.txt") as train_txt:
    train_data = train_txt.read()

train_data

'X, a tiefling with striking crimson skin and curling obsidian horns, was born during a violent storm in the town of Eldershade. His mother, a renowned herbalist named Liora, claimed that the tempest heralded his arrival as both an omen and a blessing. From an early age, X felt destined for more than the serene life of gathering herbs and tending to the local grove where she practiced her craft; he longed for adventure beyond the tranquil valley that sheltered him. As whispers of dark creatures creeping closer from the Shadow Vale reached their ears, Liora\\\'s warnings echoed in his mind: \\"The world outside is not kind to those who wield power.\\" But drawn by tales of heroism and glory spun by passing travelers at local taverns, X couldn\\\'t resist planting one foot on home soil while stretching another toward destiny.\nWhen Liora vanished one moonless night without a traceâ€”taken by dark forces known only in folkloreâ€”X\\\'s plans shifted drastically from idyllic exploration to

### Replacing the X (character name context)

In [11]:
CHARACTER_NAME = "Circle Lovelace"

train_data = train_data.replace('X', CHARACTER_NAME)

### Building the model

In [12]:
# Number of words a next word depends on
state_size = 2

markov_model = markovify.Text(
    input_text=train_data,
    state_size=state_size
)

markov_model.state_size

2

In [13]:
# (Optional) Compile to improve the model's generation speed and reduce its size
markov_model = markov_model.compile()

### Generating Sentences

In [14]:
# Generates a sentence no more than 300 characters
result = markov_model.make_short_sentence(300)

print(result)

As years passed under its influence, they honed these newfound abilities as both an omen and a blessing.


### Generating a paragraph with N sentences

In [15]:
# Generation parameters
N_SENTENCES = 5
N_CHARACTER_LIMIT = 300

results = []
for _ in range(N_SENTENCES):
    results.append(markov_model.make_short_sentence(N_CHARACTER_LIMIT))

print(" ".join(results))

Yet power has its price: each incantation drained fragments of his ancestors who were once revered for their communion with nature\'s spirits. As years passed under its influence, they honed these newfound abilities as both an omen and a blessing. Yet power has its price: each incantation drained fragments of his ancestors who were once revered for their communion with nature\'s spirits. Yet, when Circle Lovelace honed his innate abilities among woodland creatures and learned to harness magic drawn directly from nature itself. Circle Lovelace, a tiefling with striking crimson skin and curling obsidian horns, was born under a blood-red moon in the sun-drenched meadows, he found solace amidst the whispering leaves and blooming flowers.


In [16]:
# Generation parameters
N_SENTENCES = 5
min_words = 4
max_words = 25

results = []
for _ in range(N_SENTENCES):
    result = None
    # Prevent no result
    while result is None:
        result = markov_model.make_sentence(
        min_words=min_words,
        max_words=max_words
    )
    results.append(result)

print(" ".join(results))

Yet power has its price: each incantation drained fragments of his ancestors who were once revered for their communion with nature\'s spirits. As he delved deeper into this arcane knowledge, Circle Lovelace learned to harness magic drawn directly from nature itself. Cursed by an unyielding spirit, Circle Lovelace was born under a blood-red moon in the town of Eldershade. Circle Lovelace honed his innate abilities among woodland creatures and learned to harness magic drawn directly from nature itself. While other children played games in the heart of Tharindal.
